In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import os

from dotenv import load_dotenv

import pandas as pd

# keep the API keys in a `.env` file in the local root directory
load_dotenv()


True

In [9]:
from pisa.administrative_area import AdministrativeArea

timor_leste = AdministrativeArea(country_name="Timor-Leste", admin_level=1)

# these are the boundaries of Baucau
# type: Polygon
baucau = timor_leste.get_admin_area_boundaries("Baucau")

INFO:pisa.administrative_area:Validating country name: Timor-Leste
INFO:pisa.administrative_area:Country name 'Timor-Leste' validated successfully
INFO:pisa.administrative_area:Retrieving boundaries of all administrative areas of level 1 for country Timor-Leste


In [10]:
DISTANCE_TYPE = "length"

DISTANCE_VALUES = [1000]

MODE_OF_TRANSPORT = "driving"

### Coordinates of actual hospital in Timor-Leste

In [11]:

hospital_coordinates = [
        (126.60048, -8.54733),
    ]

hospital_df = pd.DataFrame(hospital_coordinates, columns=["longitude", "latitude"])

# you need (latitude, longitude) for graph_from_point(), 
# in contrast to the (longitude, latitude) construction of facilities_df

hospital_lat_lon = (hospital_df.latitude.values[0], hospital_df.longitude.values[0])
hospital_lat_lon

(-8.54733, 126.60048)

### Mapbox isopolygon

In [12]:
from pisa.isopolygons import MapboxIsopolygonCalculator

MAPBOX_API_TOKEN = os.getenv("MAPBOX_API_TOKEN")

mapbox_iso_1000 = MapboxIsopolygonCalculator(
                facilities_df=hospital_df,
                distance_type=DISTANCE_TYPE,
                distance_values=DISTANCE_VALUES,
                mode_of_transport=MODE_OF_TRANSPORT,
                mapbox_api_token=MAPBOX_API_TOKEN,
            ).calculate_isopolygons()

mapbox_iso_1000

INFO:pisa.utils:
=== Cache Status ===
INFO:pisa.utils:Function: _fetch_isopolygons
INFO:pisa.utils:Cache file: mapbox_cache/773dc0e5410fc7df3240c0d31cb8b1c63bbf6b691da6dd7acd515e9c44e2af59.pkl
INFO:pisa.utils:Cache HIT - Loading cached result


,ID_1000
0,"POLYGON ((126.60248 -8.541311, 126.601428 -8.5..."


In [13]:
from pisa.visualisation import plot_isochrones

plot_isochrones(mapbox_iso_1000, baucau)

OSM Isopolygon

In [14]:
from pisa.isopolygons import OsmIsopolygonCalculator
from pisa.osm_road_network import OsmRoadNetwork

road_network = OsmRoadNetwork(
    admin_area_boundaries=baucau,
    mode_of_transport=MODE_OF_TRANSPORT,
    distance_type=DISTANCE_TYPE,
).get_osm_road_network()

INFO:pisa.osm_road_network:OSM road network set with parameters network_type 'drive' and distance_type 'length'


In [22]:
osm_1000 = OsmIsopolygonCalculator(
    facilities_df=hospital_df,
    distance_type=DISTANCE_TYPE,
    distance_values=DISTANCE_VALUES,
    road_network=road_network,
).calculate_isopolygons()


/Users/catherine/code/Public-Infrastructure-Service-Access/pisa/isopolygons.py:238: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  disks = nodes_gdf.buffer(node_buffer).geometry
/Users/catherine/code/Public-Infrastructure-Service-Access/pisa/isopolygons.py:243: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cylinders = edges_gdf.buffer(edge_buffer).geometry


In [23]:
plot_isochrones(osm_1000, baucau)

In [17]:
from pisa.isopolygons import OsmIsopolygonCalculatorAlternative

osm_alt_1000 = OsmIsopolygonCalculatorAlternative(
    facilities_df=hospital_df,
    distance_type=DISTANCE_TYPE,
    distance_values=DISTANCE_VALUES,
    road_network=road_network,
).calculate_isopolygons()


In [18]:
plot_isochrones(osm_alt_1000, baucau)